Inference Notebook for Tool dataset
- Model path: "artisanalwasp/resized_tool_dataset_model"
- Training Dataset: "artisanalwasp/resized_tool_dataset" 

- Install dependencies

In [1]:
! pip install -U peft transformers diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 10.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


Diffusers Pipeline Setup        
Loading Stable Diffusion XL Base 1.0 as base model then attaching LoRa adapters


In [1]:
%load_ext autoreload
%autoreload 2

from diffusers import DiffusionPipeline
import torch
import os
from src.prompt_generator import PromptGenerator

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

# Optional CPU offloading to save same GPU memory
pipe.enable_model_cpu_offload()

model_path = "artisanalwasp/resized_tool_dataset_model_batchsize2"
model_name = (model_path).split("/")[-1]
print("Name of the model: ", model_name)

# Loading Trained LoRa weights
pipe.load_lora_weights(model_path)

/home/bsuetcueoglu/miniconda3/envs/thesis-all-modules-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.52it/s]


Name of the model:  resized_tool_dataset_model_batchsize2


Generate prompts for different wear severities
- Wear severities can be *low*, *moderate* or *high*

In [ ]:
total_num_images = 50
tool_type = "Insert"
wear_severities = ["Low Wear", "Moderate Wear", "High Wear"]

os.makedirs(f"./generated_images/{model_name}/not_resized/random/", exist_ok=True)

for i in range(total_num_images):
    
    prompt = PromptGenerator.generate_random_prompt_for_tool_dataset(tool_type, wear_severities)
    
    image = pipe(
        prompt=prompt,
        num_inference_steps=50,
        width=1024,
        height=1024,
        guidance_scale= 7.0 #how much to prompt effects the generated image
    ).images[0]

    image.save(f"./generated_images/{model_name}/not_resized/random/{prompt}_{i}.png")

100%|██████████| 50/50 [00:09<00:00,  5.20it/s]


Time taken to generate image 1: 11.46 seconds


Resize Generated Images to the Original Size
- Images in Tool Dataset are 2392x1144 pixels

In [7]:
import os
target_dir = "sdfvtcm"
cwd = os.getcwd()
if not cwd.endswith(target_dir):
    os.chdir(os.pardir)

from DatasetPreprocessor.src.image_preprocessor_utils import ImageUtils

generated_images_dir = "ImageGenerationwithStableDiff/generated_images/resized_tool_dataset_model_batchsize2/not_resized/random/"
resized_images_dir = "Data/Tool_dataset/generated/images"
ImageUtils.resize_all_images(generated_images_dir, resized_images_dir, "lanczos", 2392, 1144)

50 images are resized to 2392x1144 using lanczos and saved to Data/Tool_dataset/generated/images


Annotate Images Manually and Convert COCO annotations to Binary Masks

In [2]:
import os
target_dir = "sdfvtcm"
cwd = os.getcwd()
if not cwd.endswith(target_dir):
    os.chdir(os.pardir)

from DatasetPreprocessor.src.coco_to_binary_converter import COCOtoBinaryConverter

coco_ann_path = "./Data/Tool_dataset/generated/gen_10/generated_10/labels_td_10.json"
binary_masks_dir = "./Data/Tool_dataset/generated/gen_10/binary_masks_10/"
COCOtoBinaryConverter.create_masks(coco_ann_path, 1, binary_masks_dir)

coco_ann_path_20 = "./Data/Tool_dataset/generated/gen_20/generated_20/labels_td_20.json"
binary_masks_dir_20 = "./Data/Tool_dataset/generated/gen_20/binary_masks_20/"
COCOtoBinaryConverter.create_masks(coco_ann_path_20, 1, binary_masks_dir_20)

coco_ann_path_30 = "./Data/Tool_dataset/generated/gen_30/generated_30/labels_td_30.json"
binary_masks_dir_30 = "./Data/Tool_dataset/generated/gen_30/binary_masks_30/"
COCOtoBinaryConverter.create_masks(coco_ann_path_30, 1, binary_masks_dir_30)

Binary masks saved to ./Data/Tool_dataset/generated/gen_10/binary_masks_10/
Binary masks saved to ./Data/Tool_dataset/generated/gen_20/binary_masks_20/
Binary masks saved to ./Data/Tool_dataset/generated/gen_30/binary_masks_30/
